In [ ]:
pip install wikipedia-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import wikipediaapi

# Get WIKI sections and subscetions data

In [ ]:
# ADD SECTION TO IGNORE TO THE FOLLOWING CODE
SECTIONS_TO_IGNORE = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Footnotes",
    "Bibliography",
    "Sources",
    "Citations",
    "Literature",
    "Photo gallery",
    "Photos",
    "Gallery",
    "Works cited",
    "Notes",
    "Notes and references",
    "References and sources",
    "References and notes",
]

In [ ]:
def get_sections(
          section: wikipediaapi.WikipediaPageSection,
          parent_title: list[str],
          sections_to_ignore: set[str] = SECTIONS_TO_IGNORE,
      ) -> list[tuple[list[str],str]]:
 
  """Gather sections and subsections data."""

  sect_title = section.title 
  title = parent_title
  title.extend([sect_title])
  results = []
  
  if sect_title not in sections_to_ignore:
    sect_text = section.text   
    string_section = ( title , sect_text)
    results.extend([string_section])
    if len(section.sections)>1:
      for subsection in section.sections:
        get_sections(subsection, title, sections_to_ignore)     
  return results

def get_pages (
            page: wikipediaapi.WikipediaPage,
            sections_to_ignore: set[str] = SECTIONS_TO_IGNORE,
        )-> list[tuple[list[str],str]]:

    """Gather the page information: title and summary, and then go deep in sections inforamtion."""

    parent_title = page.title
    print(f"parent_title = {parent_title} ")

    summary = page.summary
    string_parent = ([parent_title], summary)
    results=[string_parent]

    if len(page.sections) > 0:
      for section in page.sections:
        results.extend(get_sections(section,[parent_title]))

    return results



wiki = wikipediaapi.Wikipedia('en')
CATEGORY_TITLE = "Category:2022 FIFA World Cup"

# Get the category object for "2022 FIFA World Cup"
cat = wiki.page(CATEGORY_TITLE)

# Print the name of the category
print("Name: ", cat.categorymembers)

# Get the articles included in this category
# # if it's an article:  wikipediaapi.Namespace.MAIN = 0
# # if it's a subcategory: wikipediaapi.Namespace.CATEGORY = 14
articles = [w for w in cat.categorymembers.values() if w.ns == wikipediaapi.Namespace.MAIN]

wiki_corpus=[]
for page in articles:
  wiki_corpus.extend(get_pages(page))
wiki_corpus[:5]

Name:  {'2022 FIFA World Cup': 2022 FIFA World Cup (id: ??, ns: 0), 'User:Monziguazini/sandbox': User:Monziguazini/sandbox (id: ??, ns: 2), '2015 FIFA corruption case': 2015 FIFA corruption case (id: ??, ns: 0), '2022 FIFA World Cup opening ceremony': 2022 FIFA World Cup opening ceremony (id: ??, ns: 0), 'Adidas Al Rihla': Adidas Al Rihla (id: ??, ns: 0), 'Battle of Lusail': Battle of Lusail (id: ??, ns: 0), '2022 FIFA World Cup broadcasting rights': 2022 FIFA World Cup broadcasting rights (id: ??, ns: 0), 'List of 2022 FIFA World Cup controversies': List of 2022 FIFA World Cup controversies (id: ??, ns: 0), 'FIFA World Cup Qatar 2022 Official Soundtrack': FIFA World Cup Qatar 2022 Official Soundtrack (id: ??, ns: 0), '2022 FIFA World Cup final': 2022 FIFA World Cup final (id: ??, ns: 0), 'Garcia Report': Garcia Report (id: ??, ns: 0), '2022 FIFA World Cup Group A': 2022 FIFA World Cup Group A (id: ??, ns: 0), '2022 FIFA World Cup Group B': 2022 FIFA World Cup Group B (id: ??, ns: 0), 

[(['2022 FIFA World Cup'],
  "The 2022 FIFA World Cup was the 22nd FIFA World Cup, the quadrennial world championship for national football teams organized by FIFA. It took place in Qatar from 20 November to 18 December 2022, after the country was awarded the hosting rights in 2010. It was the first World Cup to be held in the Arab world and Muslim world, and the second held entirely in Asia after the 2002 tournament in South Korea and Japan.This tournament was the last with 32 participating teams, with the number of teams being increased to 48 for the 2026 edition. To avoid the extremes of Qatar's hot climate, the event was held during November and December. It was held over a reduced time frame of 29 days with 64 matches played in eight venues across five cities. Qatar entered the event—their first World Cup—automatically as the host's national team, alongside 31 teams determined by the qualification process.\nArgentina were crowned the champions after winning the final against the t

# Filter short text

In [ ]:
# filter out short/blank sections
def keep_section(section: tuple[list[str], str]) -> bool:
    """Return True if the section should be kept, False otherwise."""
    titles, text = section
    if len(text) < 20:
        return False
    else:
        return True

In [ ]:
wiki_corpus_filtered = [sect for sect in wiki_corpus if keep_section(sect)]

# Split inputs to 1600 max_tokens

In [ ]:
pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.5 MB/s eta 0:00:00


In [ ]:
import tiktoken

In [ ]:
#Source code from OpenAI documentation
GPT_MODEL = "gpt-3.5-turbo"  # only matters insofar as it selects which tokenizer to use


def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def halved_by_delimiter(string: str, delimiter: str = "\n") -> list[str, str]:
    """Split a string in two, on a delimiter, trying to balance tokens on each side."""
    chunks = string.split(delimiter)
    if len(chunks) == 1:
        return [string, ""]  # no delimiter found
    elif len(chunks) == 2:
        return chunks  # no need to search for halfway point
    else:
        total_tokens = num_tokens(string)
        halfway = total_tokens // 2
        best_diff = halfway
        for i, chunk in enumerate(chunks):
            left = delimiter.join(chunks[: i + 1])
            left_tokens = num_tokens(left)
            diff = abs(halfway - left_tokens)
            if diff >= best_diff:
                break
            else:
                best_diff = diff
        left = delimiter.join(chunks[:i])
        right = delimiter.join(chunks[i:])
        return [left, right]

def truncated_string(
    string: str,
    model: str,
    max_tokens: int,
    print_warning: bool = True,
) -> str:
    """Truncate a string to a maximum number of tokens."""
    encoding = tiktoken.encoding_for_model(model)
    encoded_string = encoding.encode(string)
    truncated_string = encoding.decode(encoded_string[:max_tokens])
    if print_warning and len(encoded_string) > max_tokens:
        print(f"Warning: Truncated string from {len(encoded_string)} tokens to {max_tokens} tokens.")
    return truncated_string

def split_strings_from_subsection(
    subsection: tuple[list[str], str],
    max_tokens: int = 1000,
    model: str = GPT_MODEL,
    max_recursion: int = 5,
) -> list[str]:
    """
    Split a subsection into a list of subsections, each with no more than max_tokens.
    Each subsection is a tuple of parent titles [H1, H2, ...] and text (str).
    """
    titles, text = subsection
    string = "\n\n".join(titles + [text])
    num_tokens_in_string = num_tokens(string)
    # if length is fine, return string
    if num_tokens_in_string <= max_tokens:
        return [string]
    # if recursion hasn't found a split after X iterations, just truncate
    elif max_recursion == 0:
        return [truncated_string(string, model=model, max_tokens=max_tokens)]
    # otherwise, split in half and recurse
    else:
        titles, text = subsection
        for delimiter in ["\n\n", "\n", ". "]:
            left, right = halved_by_delimiter(text, delimiter=delimiter)
            if left == "" or right == "":
                # if either half is empty, retry with a more fine-grained delimiter
                 continue
            else:
                # recurse on each half
                results = []
                for half in [left, right]:
                    half_subsection = (titles, half)
                    half_strings = split_strings_from_subsection(
                        half_subsection,
                        max_tokens=max_tokens,
                        model=model,
                        max_recursion=max_recursion - 1,
                    )
                    results.extend(half_strings)
                return results
    # otherwise no split was found, so just truncate (should be very rare)
    return [truncated_string(string, model=model, max_tokens=max_tokens)]

In [ ]:
# split sections into chunks
MAX_TOKENS = 1600
wiki_corpus_final = []
for section in wiki_corpus_filtered:
    wiki_corpus_final.extend(split_strings_from_subsection(section, max_tokens=MAX_TOKENS))

print(f"{len(wiki_corpus_filtered)} Wikipedia sections split into {len(wiki_corpus_final)} strings.")

109 Wikipedia sections split into 109 strings.


In [ ]:
wiki_corpus_final[:2]

["2022 FIFA World Cup\n\nThe 2022 FIFA World Cup was the 22nd FIFA World Cup, the quadrennial world championship for national football teams organized by FIFA. It took place in Qatar from 20 November to 18 December 2022, after the country was awarded the hosting rights in 2010. It was the first World Cup to be held in the Arab world and Muslim world, and the second held entirely in Asia after the 2002 tournament in South Korea and Japan.This tournament was the last with 32 participating teams, with the number of teams being increased to 48 for the 2026 edition. To avoid the extremes of Qatar's hot climate, the event was held during November and December. It was held over a reduced time frame of 29 days with 64 matches played in eight venues across five cities. Qatar entered the event—their first World Cup—automatically as the host's national team, alongside 31 teams determined by the qualification process.\nArgentina were crowned the champions after winning the final against the title 

# Embeddings

In [ ]:
pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 28.3 MB/s eta 0:00:00


In [ ]:
import openai

openai.api_key="YOUR_API_KEY"

In [ ]:
# As we have a small corpus, 109 inputs, we will be using embedding directly. 
# Otherwise you can use the code in the following cell, to split the corpus on small batchs
# you can submit up to 2048 embedding inputs per request

import pandas as pd

# calculate embeddings
EMBEDDING_MODEL = "text-embedding-ada-002" 

response = openai.Embedding.create(
              model=EMBEDDING_MODEL, 
              input=wiki_corpus_final
              )

embeddings = [e["embedding"] for e in response["data"]]

print(f"length of embeddings array for the whole corpus {len(embeddings)}")
print(f"each embedding has a vector length of {len(embeddings[0])}")

df = pd.DataFrame({"text": wiki_corpus_final, "embedding": embeddings})
df.head()

length of embeddings array for the whole corpus 109
each embedding has a vector length of 1536


,text,embedding
0,2022 FIFA World Cup\n\nThe 2022 FIFA World Cup...,"[-0.002434000140056014, -0.012627567164599895,..."
1,2022 FIFA World Cup\n\nOverview\n\nSchedule\n\...,"[-0.0006040519219823182, -0.020295508205890656..."
2,2022 FIFA World Cup\n\nHost selection\n\nHost ...,"[0.01563253253698349, -0.009458768181502819, 0..."
3,2022 FIFA World Cup\n\nVenues\n\nThe first fiv...,"[0.01284846756607294, -0.007050588261336088, 0..."
4,2022 FIFA World Cup\n\nSquads\n\nBefore submit...,"[0.014988874085247517, -0.0017697860021144152,..."


In [ ]:
#SPLIT THE DATA INTO BACTHS oF LENGTH 1000, TO CALL AT EACH TIME
import pandas as pd

# calculate embeddings
EMBEDDING_MODEL = "text-embedding-ada-002"  # OpenAI's best embeddings as of Apr 2023
BATCH_SIZE = 1000  # you can submit up to 2048 embedding inputs per request

embeddings = []
for batch_start in range(0, len(wiki_corpus_final), BATCH_SIZE):
    batch_end = batch_start + BATCH_SIZE
    batch = wiki_corpus_final[batch_start:batch_end]
    print(f"Batch {batch_start} to {batch_end-1}")
    response = openai.Embedding.create(model=EMBEDDING_MODEL, input=batch)
    for i, be in enumerate(response["data"]):
        assert i == be["index"]  # double check embeddings are in same order as input
    batch_embeddings = [e["embedding"] for e in response["data"]]
    embeddings.extend(batch_embeddings)

df = pd.DataFrame({"text": wiki_corpus_final, "embedding": embeddings})

Batch 0 to 999


# Save dataset

In [ ]:
df.to_csv("embeddings_2022_fifa_world_cup.csv",index=False)